In [2]:
import pandas as pd
import numpy as np

def load_excel(path:str) -> pd.DataFrame:
    data = pd.read_csv(path)
    df = pd.DataFrame(data)

    # Chuẩn hóa cột current_price
    df["current_price"] = (
        df["current_price"]
        .astype(str)                                     # Đảm bảo dạng chuỗi
        .str.replace(r"[^\d]", "", regex=True)           # Chỉ giữ số
        .replace("", np.nan)                             # Chuỗi rỗng -> NaN
    )

    # Chuyển sang số (bỏ qua lỗi), rồi thêm hậu tố "vnd"
    df["current_price"] = df["current_price"].astype(float).dropna().astype("Int64").astype(str) + " vnd"

    return df
df = load_excel('hoanghamobile.csv')
df.head()

,_id,url,title,product_promotion,product_specs,current_price,color_options
0,666baeb49793e149fe7393b4,https://hoanghamobile.com/dien-thoai/nokia-321...,nokia 3210 4g - chính hãng,NaN,Công nghệ màn hình:\r\nIPS<br> Kích thước màn ...,1590000 vnd,"['Màu Vàng', 'Xanh', 'Màu Đen']"
1,666baeb49793e149fe7393bc,https://hoanghamobile.com/dien-thoai-di-dong/s...,samsung galaxy a05s - 6gb/128gb (bhđt),- Ưu đãi trả góp 0% qua Shinhan Finance hoặc M...,"Công nghệ màn hình:\r\nPLS LCD, 90Hz<br> Độ ph...",3490000 vnd,"['Màu Đen', 'Xanh', 'Bạc']"
2,666baeb49793e149fe7393b8,https://hoanghamobile.com/dien-thoai/vivo-y03-...,vivo y03 4/64gb- chính hãng,- Ưu đãi tặng Sim Mobifone Hera & Key bản quyề...,Công nghệ màn hình:\r\nIPS LCD<br> Độ phân giả...,2790000 vnd,"['Màu Đen', 'Xanh']"
3,666baeb49793e149fe7393b5,https://hoanghamobile.com/dien-thoai/nokia-220...,nokia 220 4g - chính hãng,NaN,Công nghệ màn hình:\r\nMàn hình LCD<br> Kích t...,990000 vnd,"['Màu Đen', 'Cam đào']"
4,666baeb49793e149fe7393b9,https://hoanghamobile.com/dien-thoai/xiaomi-re...,điện thoại xiaomi redmi 12 4gb/128gb,- Ưu đãi trả góp 0% qua Shinhan Finance hoặc M...,Độ phân giải:\r\nFull HD<br> Cổng kết nối:\r\n...,2990000 vnd,"['Màu Đen', 'Bạc', 'Xanh Dương']"


In [3]:
import pandas as pd
import re

def convert_table_to_rows(df: pd.DataFrame) -> list:
    """
    Chuyển đổi DataFrame (bảng) thành một danh sách các chuỗi.
    Mỗi chuỗi đại diện cho một hàng, với các giá trị được nối lại với nhau.

    Args:
        df (pd.DataFrame): DataFrame đầu vào.

    Returns:
        list: Danh sách các chuỗi, mỗi chuỗi là một hàng.
    """
    result_list = []
    for index, row in df.iterrows():
        # Chuyển tất cả giá trị trong hàng thành chuỗi và nối chúng lại với nhau
        row_string = ", ".join(str(value) for value in row)
        result_list.append(row_string)
        
    cleaned_text = [" ".join(row_string.replace("<br>", "").split()) for row_string in result_list if row_string.strip() != ""]
    result_list = [re.sub(r'<[^>]*>|\s+', ' ', row_string).strip() for row_string in cleaned_text]
    
    return result_list


result_list = convert_table_to_rows(df)
result_list

["666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng, nan, Công nghệ màn hình: IPS Kích thước màn hình: 2.4 inch Độ phân giải: 2MP Hệ điều hành: S30+ Bộ nhớ trong: 128MB3 RAM: 64MB Mạng di động: 2G, 3G, 4G, Hỗ trợ VoLTE2 Số khe SIM: Hai SIM Nano SIM + Nano SIM Dung lượng pin: 1450mAh, 1590000 vnd, ['Màu Vàng', 'Xanh', 'Màu Đen']",
 "666baeb49793e149fe7393bc, https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-a05s-6gb-128gb-bh%C4%91t, samsung galaxy a05s - 6gb/128gb (bhđt), - Ưu đãi trả góp 0% qua Shinhan Finance hoặc Mirae Asset Finance- Giảm 5% không giới hạn khuyến mãi qua Homepaylater- Giảm thêm tới 700.000đ khi thanh toán qua Kredivo.- Giảm 50% tối đa 700k khi mở thẻ tín dụng Vpbank trên SenID- Giảm 20% tối đa 500k khi mở thẻ tín dụng TPBank EVO- Mở thẻ tín dụng VIB - Nhận Voucher 600.000đ- Giảm 1% tối đa 100.000đ khi thanh toán qua Zalopay, Công nghệ màn hình: PLS LCD, 90Hz Độ phân giải: FHD+ (2400 x 1080),

# char specials


In [4]:
import re

def clean_text(text):
    """
    Hàm loại bỏ ký tự đặc biệt trong văn bản,
    loại bỏ các tag HTML,
    nhưng vẫn giữ nguyên link và ID (token đầu tiên).
    """
    if not text:
        return text

    # 1. Loại bỏ tag HTML (vd: <br>, <div>...</div>)
    text = re.sub(r'<.*?>', ',', text)

    # 2. Tách token
    tokens = text.strip().split(',')
    if not tokens:
        return text

    # Giữ lại ID (token đầu tiên)
    cleaned_tokens = [tokens[0]] + [tokens[1]] + [tokens[2]]

    # Regex nhận diện link (http, https, www, .com, .vn, ...)
    url_pattern = re.compile(r'(https?://\S+|www\.\S+|\S+\.(com|vn|net|org)\S*)')

    for token in tokens[3:]:
        if url_pattern.match(token):  
            cleaned_tokens.append(token.strip())  # Giữ nguyên link
        else:
            # Chỉ giữ chữ cái, số và dấu cách
            cleaned = re.sub(r'[^0-9a-zA-ZÀ-Ỹà-ỹ.\s]', '', token.strip())
            if cleaned:  # Bỏ token rỗng
                cleaned_tokens.append(cleaned.strip())

    return ",".join(cleaned_tokens).strip()

# Ví dụ sử dụng
sample = clean_text(result_list[0])
print(sample)

666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng,nan,Công nghệ màn hình IPS Kích thước màn hình 2.4 inch Độ phân giải 2MP Hệ điều hành S30 Bộ nhớ trong 128MB3 RAM 64MB Mạng di động 2G,3G,4G,Hỗ trợ VoLTE2 Số khe SIM Hai SIM Nano SIM  Nano SIM Dung lượng pin 1450mAh,1590000 vnd,Màu Vàng,Xanh,Màu Đen


In [5]:
from underthesea import word_tokenize
import os
import numpy as np

text = sample

def remove_stopwords_vi(text: str = 'inputs là đoạn văn bản cụ thể của bạn' , path_documents_vi:str='stopwords-vietnamese.txt'):
    id_str = text.split(',')[0] + ','
    link_https_str = text.split(',')[1] + ','
    name_object = text.split(',')[2] + ','
    # print(id_str, link_https_str, name_object)
    
    text = ','.join(text.split(',')[3:])
    stop_words = set(open(str(path_documents_vi), encoding="utf-8").read().splitlines())
    tokens = str(word_tokenize(text, format="text")).split(',')
    filtered = [w.strip() for w in tokens if w.lower() not in stop_words]
    return id_str + link_https_str + name_object + ', '.join(filtered)

text = remove_stopwords_vi(text)

text

'666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng,nan, Công_nghệ màn_hình IPS_Kích_thước màn_hình 2.4 inch_Độ phân_giải 2MP_Hệ điều_hành S30 Bộ_nhớ trong 128MB3_RAM 64MB_Mạng di_động 2G, 3G, 4G, Hỗ_trợ VoLTE2_Số khe SIM Hai_SIM Nano_SIM Nano_SIM Dung_lượng pin 1450 mAh, 1590000 vnd, Màu_Vàng, Xanh, Màu Đen'

In [6]:
import re

def clean_text(text):
    """
    Làm sạch văn bản:
    - Giữ nguyên ID (token đầu tiên) và link (https, http, www, .com, .vn...).
    - Xóa ký tự đặc biệt, xuống dòng (\r, \n).
    - Xóa toàn bộ thẻ HTML (<br>, <div>...</div>, ...).
    - Thay thế '_' thành khoảng trắng.
    - Chuẩn hóa khoảng trắng.
    """
    if not text:
        return text

    # 1. Xóa thẻ HTML
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Xóa ký tự xuống dòng và tab
    text = text.replace("\r", " ").replace("\n", " ").replace("\t", " ")

    # 3. Thay thế dấu "_" thành khoảng trắng
    text = text.replace("_", " ")

    # 4. Tách token
    tokens = text.split(',')
    if not tokens:
        return text

    # Giữ lại ID (token đầu tiên)
    cleaned_tokens = [tokens[0]+',' + tokens[1]+',' + tokens[2] +',']

    # Regex nhận diện link (http, https, www, .com, .vn, .net, .org...)
    url_pattern = re.compile(r'(https?://\S+|www\.\S+|\S+\.(com|vn|net|org)\S*)')

    for token in tokens[3:]:
        if url_pattern.match(token):
            cleaned_tokens.append(token + ', ')  # Giữ nguyên link
        else:
            # Giữ chữ cái + số + khoảng trắng (loại bỏ ký tự đặc biệt)
            cleaned = re.sub(r'[^0-9a-zA-ZÀ-Ỹà-ỹ\s]', '', token)
            if cleaned:
                cleaned_tokens.append(cleaned + ',')

    # 5. Ghép lại và chuẩn hóa khoảng trắng
    return re.sub(r'\s+', ' ', " ".join(cleaned_tokens)).strip()

text = '''666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng,nan, Công_nghệ màn_hình IPS_Kích_thước màn_hình 2.4 inch_Độ phân_giải 2MP_Hệ điều_hành S30 Bộ_nhớ trong 128MB3_RAM 64MB_Mạng di_động 2G, 3G, 4G, Hỗ_trợ VoLTE2_Số khe SIM Hai_SIM Nano_SIM Nano_SIM Dung_lượng pin 1450 mAh, 1590000 vnd, Màu_Vàng, Xanh, Màu Đen'''
text = clean_text(text)
text


'666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng, nan, Công nghệ màn hình IPS Kích thước màn hình 24 inch Độ phân giải 2MP Hệ điều hành S30 Bộ nhớ trong 128MB3 RAM 64MB Mạng di động 2G, 3G, 4G, Hỗ trợ VoLTE2 Số khe SIM Hai SIM Nano SIM Nano SIM Dung lượng pin 1450 mAh, 1590000 vnd, Màu Vàng, Xanh, Màu Đen,'

# process sequences


In [7]:
import re

def normalize_record(text, fix_inch_heu=False):
    """
    Normalize a product text line:
    - remove extra commas, trailing commas
    - remove token 'nan' (case-insensitive)
    - replace underscores, remove HTML/newlines if any
    - normalize units: MB, GB, MP, mAh, 4G
    - collapse adjacent duplicate tokens (case-insensitive)
    - optionally apply heuristic to fix large 'inch' numbers (fix_inch_heu=True will convert e.g. 667->6.67 if detected)
    """
    if not text:
        return text

    # 1. basic cleanup
    text = re.sub(r'<.*?>', ' ', text)               # remove html tags
    text = text.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')
    text = text.replace('_', ' ')
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. remove standalone 'nan'
    text = re.sub(r'\b[nN][aA][nN]\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. remove commas that are not in urls (simple approach: replace comma with space)
    #    keep url/ID intact by doing token-wise handling later
    # We'll split tokens and treat first token (ID) and tokens that match URL pattern specially.
    tokens = text.split()

    if not tokens:
        return text

    # Save leading ID (first token) and keep URLs intact
    id_token = tokens[0]
    url_pattern = re.compile(r'^(https?://\S+|www\.\S+|\S+\.(com|vn|net|org)\S*)$', flags=re.IGNORECASE)

    cleaned = [id_token]
    prev_tok_lower = None

    for tok in tokens[1:]:
        # keep url as-is
        if url_pattern.match(tok):
            cur = tok
        else:
            # remove commas / trailing punctuation
            cur = re.sub(r'[,\u200b]+', ' ', tok)
            cur = re.sub(r'^[\W_]+|[\W_]+$', '', cur)  # trim non-word at ends
            cur = cur.strip()

            # normalize units
            cur = re.sub(r'(?i)\b(\d+)\s*[gG][bB]\b', r'\1 GB', cur)
            cur = re.sub(r'(?i)\b(\d+)\s*[mM][bB]\b', r'\1 MB', cur)
            cur = re.sub(r'(?i)\b(\d+)\s*[mM][pP]\b', r'\1 MP', cur)
            cur = re.sub(r'(?i)\b(\d+)\s*[mM][aA][hH]\b', r'\1 mAh', cur)
            cur = re.sub(r'(?i)\b(\d+)\s*[gG]\b', r'\1G', cur)  # 4G

            # fix patterns like '128MB3' -> '128 MB' (remove stray trailing digits after unit)
            cur = re.sub(r'(\b\d+\s*(?:MB|GB|MP|mAh))3\b', r'\1', cur)

            # normalize common tokens
            cur = re.sub(r'(?i)\bips\s*lcd\b', 'IPS LCD', cur)
            cur = re.sub(r'(?i)\bfull\s*hd\s*\+?\b', 'Full HD+', cur)
            cur = re.sub(r'(?i)\bsnapdragon\s*(\d+)\s*g\b', lambda m: f"Snapdragon {m.group(1)}G", cur)

            # optional heuristic fix for inch numbers that are clearly too large for phones
            if fix_inch_heu:
                m = re.match(r'^(\d{2,3})\s*(?:inch|in)$', cur, flags=re.IGNORECASE)
                if m:
                    val = int(m.group(1))
                    if 50 <= val <= 999:
                        cur = f"{val/100:.2f} inch"

        if not cur:
            continue

        # collapse adjacent duplicate tokens (case-insensitive)
        if prev_tok_lower is not None and cur.lower() == prev_tok_lower:
            continue

        cleaned.append(cur)
        prev_tok_lower = cur.lower()

    # join and final normalize spaces and punctuation
    out = ' '.join(cleaned)
    out = re.sub(r'\s+,', ',', out)
    out = re.sub(r'\s+', ' ', out).strip()
    # remove trailing commas
    out = re.sub(r',\s*$', '', out)

    return out

print(normalize_record(text, fix_inch_heu=False))


666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4G chính hãng Công nghệ màn hình IPS Kích thước màn hình 24 inch Độ phân giải 2 MP Hệ điều hành S30 Bộ nhớ trong 128MB RAM 64 MB Mạng di động 2G 3G 4G Hỗ trợ VoLTE2 Số khe SIM Hai SIM Nano SIM Nano SIM Dung lượng pin 1450 mAh 1590000 vnd Màu Vàng Xanh Màu Đen


# LOAD DATA


In [8]:

# %pip install --upgrade numpy pandas
from underthesea import text_normalize
import pandas as pd
import re

df = load_excel('hoanghamobile.csv')
result_list = convert_table_to_rows(df)
# result_list = [
#     sequence.split(',')[0] + sequence.split(',')[1] + sequence.split(',')[2] +
#     str(text_normalize(', '.join(sequence.split(',')[3:]))).lower()
#     for sequence in result_list ]
print(2)
print(np.array(result_list))
result_list = [remove_stopwords_vi(sequence) for sequence in result_list]
result_list = [clean_text(sequence) for sequence in result_list]
result_list = [normalize_record(sequence, fix_inch_heu=False) for sequence in result_list]
result_list

2
["666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng, nan, Công nghệ màn hình: IPS Kích thước màn hình: 2.4 inch Độ phân giải: 2MP Hệ điều hành: S30+ Bộ nhớ trong: 128MB3 RAM: 64MB Mạng di động: 2G, 3G, 4G, Hỗ trợ VoLTE2 Số khe SIM: Hai SIM Nano SIM + Nano SIM Dung lượng pin: 1450mAh, 1590000 vnd, ['Màu Vàng', 'Xanh', 'Màu Đen']"
 "666baeb49793e149fe7393bc, https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-a05s-6gb-128gb-bh%C4%91t, samsung galaxy a05s - 6gb/128gb (bhđt), - Ưu đãi trả góp 0% qua Shinhan Finance hoặc Mirae Asset Finance- Giảm 5% không giới hạn khuyến mãi qua Homepaylater- Giảm thêm tới 700.000đ khi thanh toán qua Kredivo.- Giảm 50% tối đa 700k khi mở thẻ tín dụng Vpbank trên SenID- Giảm 20% tối đa 500k khi mở thẻ tín dụng TPBank EVO- Mở thẻ tín dụng VIB - Nhận Voucher 600.000đ- Giảm 1% tối đa 100.000đ khi thanh toán qua Zalopay, Công nghệ màn hình: PLS LCD, 90Hz Độ phân giải: FHD+ (2400 x 1080)

['666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4G chính hãng Công nghệ màn hình IPS Kích thước màn hình 24 inch Độ phân giải 2 MP Hệ điều hành S30 Bộ nhớ trong 128MB RAM 64 MB Mạng di động 2G 3G 4G Hỗ trợ VoLTE2 Số khe SIM Hai SIM Nano SIM Nano SIM Dung lượng pin 1450 mAh 1590000 vnd Màu Vàng Xanh Màu Đen',
 '666baeb49793e149fe7393bc, https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-a05s-6gb-128gb-bh%C4%91t, samsung galaxy a05s 6 GB/128 GB bhđt Ưu đãi trả góp 0 qua Shinhan Finance hoặc Mirae Asset Finance Giảm 5 không giới hạn khuyến mãi qua Homepaylater Giảm thêm tới 700000 đ khi thanh toán qua Kredivo Giảm 50 tối đa 700 k khi mở thẻ tín dụng Vpbank trên SenID Giảm 20 tối đa 500 k khi mở thẻ tín dụng TPBank EVO Mở thẻ tín dụng VIB Nhận Voucher 600000 đ Giảm 1 tối đa 100000 đ khi thanh toán qua Zalopay Công nghệ màn hình PLS LCD 90H z Độ phân giải FHD 2400 x 1080 50 MP F1 8 AF 2 MP F2 4 2 MP F2 4 13 MP F2 0 Kích thước mà

In [9]:
len(result_list)

320

In [10]:
result_list

['666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4G chính hãng Công nghệ màn hình IPS Kích thước màn hình 24 inch Độ phân giải 2 MP Hệ điều hành S30 Bộ nhớ trong 128MB RAM 64 MB Mạng di động 2G 3G 4G Hỗ trợ VoLTE2 Số khe SIM Hai SIM Nano SIM Nano SIM Dung lượng pin 1450 mAh 1590000 vnd Màu Vàng Xanh Màu Đen',
 '666baeb49793e149fe7393bc, https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-a05s-6gb-128gb-bh%C4%91t, samsung galaxy a05s 6 GB/128 GB bhđt Ưu đãi trả góp 0 qua Shinhan Finance hoặc Mirae Asset Finance Giảm 5 không giới hạn khuyến mãi qua Homepaylater Giảm thêm tới 700000 đ khi thanh toán qua Kredivo Giảm 50 tối đa 700 k khi mở thẻ tín dụng Vpbank trên SenID Giảm 20 tối đa 500 k khi mở thẻ tín dụng TPBank EVO Mở thẻ tín dụng VIB Nhận Voucher 600000 đ Giảm 1 tối đa 100000 đ khi thanh toán qua Zalopay Công nghệ màn hình PLS LCD 90H z Độ phân giải FHD 2400 x 1080 50 MP F1 8 AF 2 MP F2 4 2 MP F2 4 13 MP F2 0 Kích thước mà

In [11]:
result_list

['666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4G chính hãng Công nghệ màn hình IPS Kích thước màn hình 24 inch Độ phân giải 2 MP Hệ điều hành S30 Bộ nhớ trong 128MB RAM 64 MB Mạng di động 2G 3G 4G Hỗ trợ VoLTE2 Số khe SIM Hai SIM Nano SIM Nano SIM Dung lượng pin 1450 mAh 1590000 vnd Màu Vàng Xanh Màu Đen',
 '666baeb49793e149fe7393bc, https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-a05s-6gb-128gb-bh%C4%91t, samsung galaxy a05s 6 GB/128 GB bhđt Ưu đãi trả góp 0 qua Shinhan Finance hoặc Mirae Asset Finance Giảm 5 không giới hạn khuyến mãi qua Homepaylater Giảm thêm tới 700000 đ khi thanh toán qua Kredivo Giảm 50 tối đa 700 k khi mở thẻ tín dụng Vpbank trên SenID Giảm 20 tối đa 500 k khi mở thẻ tín dụng TPBank EVO Mở thẻ tín dụng VIB Nhận Voucher 600000 đ Giảm 1 tối đa 100000 đ khi thanh toán qua Zalopay Công nghệ màn hình PLS LCD 90H z Độ phân giải FHD 2400 x 1080 50 MP F1 8 AF 2 MP F2 4 2 MP F2 4 13 MP F2 0 Kích thước mà

In [9]:
import random
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device="cuda",
    model_kwargs={"torch_dtype": "bfloat16"}
)

# MODELS EMBEDDING 1
document_embeddings = model.encode(result_list)

`torch_dtype` is deprecated! Use `dtype` instead!


In [10]:
text = 'Có iphone 14 hong bạn'

emb_query = model.encode([text])

In [11]:
similarity = model.similarity(emb_query, document_embeddings)
top_similarity_idx = np.argsort(-similarity.numpy().ravel())
top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]


for i in top_similarity_idx:
    print(result_list[i])

666baeb89793e149fe739479, https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-14-128gb-chinh-hang-vn-a, điện thoại iphone 14 128 GB chính hãng vn/a KM 1 Ưu đãi mua combo 3 món củ sạc dán màn hình ốp lưng Mophie Zagg chỉ 985000 đ KM 2 Ưu đãi trả góp 0 qua thẻ tín dụng Công nghệ màn hình Super Retina XDR Độ phân giải 2532 x 1170 12 MP x 12 MP Kích thước màn hình 61 inch Hệ điều hành iOS 16 Vi xử lý A15 Bionic Bộ nhớ trong 128 GB RAM 6 GB Mạng di động 5G sub 6 GHz và mmWave với 4x4 MIMO8 Gigabit LTE với 4x4 MIMO và LAA8 Chip băng thông siêu rộng cho nhận thức về không gian NFC với chế độ đọc Thẻ Express có dự trữ năng lượng Số khe SIM 1 eSIM 1 SIM vật lý 16290000 vnd Midnight Blue Purple Red Yellow Starlight
666baeb89793e149fe739478, https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-14-plus-128gb-chinh-hang-vn-a, điện thoại iphone 14 plus 128 GB chính hãng vn/a KM 1 Ưu đãi trả góp 0 qua thẻ tín dụng Công nghệ màn hình OLED Độ phân giải Super Retina XDR 1284 x 2778 Pixels 2 c

In [ ]:
import random
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device="cuda",
    model_kwargs={"torch_dtype": "bfloat16"}
)

# MODELS EMBEDDING 1
document_embeddings = model.encode(result_list)

# MODELS EMBEDDING 2
tokenized_corpus = [doc.split(" ") for doc in result_list]
bm25 = BM25Okapi(tokenized_corpus)


# --------------------------
# Prompt templates
# --------------------------
prompts = [
    "{product_name} có không shop?",
    "{product_name} chip gì vậy?",
    "camera con {product_name} thế nào vậy ?",
    "Cho mình hỏi {product_name} được bao nhiêu gb ram vậy?",
    "{product_name} còn hàng không?",
    "{product_name} có màu nào và giá bao nhiêu vậy?",
]

def get_random_prompt(product_name: str) -> str:
    return random.choice(prompts).format(product_name=product_name)

# --------------------------
# Stub search (replace later)
# --------------------------
def hybrid_search(prompt: str, k: int):
    """
    Trả về list top-k kết quả (dict với _id).
    Bạn sẽ thay bằng search thực tế.
    """
    query_embeddings = model.encode([prompt], prompt_name="query")

    if k > 5 and k % 2 == 0:
        # Compute the (cosine) similarity between the query and document embeddings
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(-similarity.numpy().ravel())
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        tokenized_query = prompt.split(" ")
        result_k = list(bm25.get_top_n(tokenized_query, result_list, n=k))
    
        result_bm25 = [result_k[idx].split()[0] for idx in range(k//2)]
        result_qwen = [np.array(top_similarity_id_products)[_,1] for _ in range(k//2)]
        
        result_combined = result_qwen + result_bm25
        result_combined = [{"_id": result_combined[idx].split()[0]} for idx in range(k)]
        
    else:
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(similarity.numpy().ravel())[:][::-1]
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        result_qwen = [{"_id": np.array(top_similarity_id_products)[_,1]} for _ in range(k)]
        result_combined = result_qwen
    
    return result_combined

def hybrid_search(prompt: str, k: int):
    """
    Trả về list top-k kết quả (dict với _id).
    Bạn sẽ thay bằng search thực tế.
    """
    query_embeddings = model.encode([prompt], prompt_name="query")

    if k > 5 and k % 2 == 0:
        # Compute the (cosine) similarity between the query and document embeddings
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(-similarity.numpy().ravel())
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        tokenized_query = prompt.split(" ")
        result_k = list(bm25.get_top_n(tokenized_query, result_list, n=k))
    
        result_bm25 = [result_k[idx].split()[0] for idx in range(k//2)]
        result_qwen = [np.array(top_similarity_id_products)[_,1] for _ in range(k//2)]
        
        result_combined = [{"_id": np.array(top_similarity_id_products)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        # print(result_combined)
        # result_combined = [{"_id": np.array(result_combined)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        
    else:
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(-similarity.numpy().ravel())
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        result_qwen = [{"_id": np.array(top_similarity_id_products)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        result_combined = result_qwen
    
    return result_combined
    

# --------------------------
# Benchmark for dataframe
# --------------------------
def benchmark_df(df: pd.DataFrame, ks=(1, 5, 10)):
    results = {f"hit@{k}": 0 for k in ks}
    total = len(df)

    for _, row in df.iterrows():
        ground_truth_id = row["_id"]
        # nếu bạn muốn lấy tên từ "title", thì thay "product_name" bằng "title"
        product_name = ' '.join(row["title"].split('-')[:-1])
        # print(product_name)
        query = get_random_prompt(str(product_name).lower())
        # print(f"[TEST] id={ground_truth_id} | query='{query}'")
        
        for k in ks:
            retrieved = hybrid_search(query, k)
            id_list = [item["_id"].strip(",") for item in retrieved]
            if ground_truth_id in id_list:
                results[f"hit@{k}"] += 1
                # print(f'{results[f"hit@{k}"]} [TRUE] {ground_truth_id} {k} {query} {retrieved}')
            # else:
                # print(f'{results[f"hit@{k}"]} [FALSE] {ground_truth_id} {k, query} {retrieved}')
            # if any(r["_id"] == ground_truth_id for r in retrieved):
            #     results[f"hit@{k}"] += 1
        
        # for k in ks:
        #     id_list = [item["_id"].strip(",") for item in retrieved]
        #     if ground_truth_id in id_list:
        #         print(f'[TRUE] {results[f"hit@{k}"]} {ground_truth_id}, {query}')
        #     else:
        #         print('[FALSE]', results[f"hit@{k}"], ground_truth_id, query, retrieved)

    for k in ks:
        results[f"hit@{k}"] /= total # pyright: ignore[reportArgumentType]

    return results

# CSV thực tế
df = pd.read_csv("hoanghamobile.csv")

evals = []

for i in range(50):
    scores = benchmark_df(df, ks=(1, 5, 10))
    evals.append([scores['hit@1'],scores['hit@5'],scores['hit@10']])
    print(f'round {i}: {scores}')

round 0: {'hit@1': 0.9625, 'hit@5': 0.996875, 'hit@10': 0.996875}
round 1: {'hit@1': 0.940625, 'hit@5': 0.996875, 'hit@10': 1.0}
round 2: {'hit@1': 0.959375, 'hit@5': 0.996875, 'hit@10': 1.0}
round 3: {'hit@1': 0.94375, 'hit@5': 1.0, 'hit@10': 1.0}
round 4: {'hit@1': 0.959375, 'hit@5': 0.996875, 'hit@10': 1.0}
round 5: {'hit@1': 0.940625, 'hit@5': 1.0, 'hit@10': 1.0}
round 6: {'hit@1': 0.9625, 'hit@5': 1.0, 'hit@10': 1.0}
round 7: {'hit@1': 0.959375, 'hit@5': 0.996875, 'hit@10': 1.0}
round 8: {'hit@1': 0.946875, 'hit@5': 1.0, 'hit@10': 1.0}
round 9: {'hit@1': 0.95625, 'hit@5': 0.996875, 'hit@10': 1.0}
round 10: {'hit@1': 0.959375, 'hit@5': 1.0, 'hit@10': 1.0}
round 11: {'hit@1': 0.95, 'hit@5': 0.996875, 'hit@10': 1.0}
round 12: {'hit@1': 0.95625, 'hit@5': 1.0, 'hit@10': 1.0}
round 13: {'hit@1': 0.946875, 'hit@5': 1.0, 'hit@10': 1.0}
round 14: {'hit@1': 0.953125, 'hit@5': 0.996875, 'hit@10': 1.0}


KeyboardInterrupt: 

In [ ]:
import numpy as np

# Chuyển list -> numpy array
evals_arr = np.array(evals)

# Tính trung bình theo cột (axis=0)
mean_scores = np.mean(evals_arr, axis=0)

print("Trung bình hit@1:", mean_scores[0])
print("Trung bình hit@5:", mean_scores[1])
print("Trung bình hit@10:", mean_scores[2])


In [ ]:
import random
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device="cuda",
    model_kwargs={"torch_dtype": "bfloat16"}
)

# MODELS EMBEDDING 1
document_embeddings = model.encode(result_list)

# MODELS EMBEDDING 2
tokenized_corpus = [doc.split(" ") for doc in result_list]
bm25 = BM25Okapi(tokenized_corpus)


# --------------------------
# Prompt templates
# --------------------------
prompts = [
    "{product_name} có không shop?",
    "{product_name} chip gì vậy?",
    "camera con {product_name} thế nào vậy ?",
    "Cho mình hỏi {product_name} được bao nhiêu gb ram vậy?",
    "{product_name} còn hàng không?",
    "{product_name} có màu nào và giá bao nhiêu vậy?",
]

def get_random_prompt(product_name: str) -> str:
    return random.choice(prompts).format(product_name=product_name)

# --------------------------
# Stub search (replace later)
# --------------------------
def hybrid_search(prompt: str, k: int):
    """
    Trả về list top-k kết quả (dict với _id).
    Bạn sẽ thay bằng search thực tế.
    """
    query_embeddings = model.encode([prompt], prompt_name="query")

    if k > 5 and k % 2 == 0:
        # Compute the (cosine) similarity between the query and document embeddings
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(similarity.numpy().ravel())[:][::-1]
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        tokenized_query = prompt.split(" ")
        result_k = list(bm25.get_top_n(tokenized_query, result_list, n=k))
    
        result_bm25 = [result_k[idx].split()[0] for idx in range(k//2)]
        result_qwen = [np.array(top_similarity_id_products)[_,1] for _ in range(k//2)]
        
        result_combined = result_qwen + result_bm25
        result_combined = [{"_id": result_combined[idx].split()[0]} for idx in range(k)]
        
    else:
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(similarity.numpy().ravel())[:][::-1]
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        result_qwen = [{"_id": np.array(top_similarity_id_products)[_,1]} for _ in range(k)]
        result_combined = result_qwen
    
    return result_combined

def hybrid_search(prompt: str, k: int):
    """
    Trả về list top-k kết quả (dict với _id).
    Bạn sẽ thay bằng search thực tế.
    """
    query_embeddings = model.encode([prompt], prompt_name="query")

    if k > 5 and k % 2 == 0:
        # Compute the (cosine) similarity between the query and document embeddings
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(-similarity.numpy().ravel())
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        tokenized_query = prompt.split(" ")
        result_k = list(bm25.get_top_n(tokenized_query, result_list, n=k))

        # print(bm25.get_top_n(tokenized_query, result_list, n=k))
        
        result_bm25 = [result_k[idx].split()[0] for idx in range(k//2)]
        result_qwen = [np.array(top_similarity_id_products)[_,1] for _ in range(k//2)]
        
        result_combined = result_qwen + result_bm25
        
        result_combined = [{"_id": result_combined[_], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        # print(result_combined)
        # result_combined = [{"_id": np.array(result_combined)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        
    else:
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(-similarity.numpy().ravel())
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        result_qwen = [{"_id": np.array(top_similarity_id_products)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        result_combined = result_qwen
    
    return result_combined
    

# --------------------------
# Benchmark for dataframe
# --------------------------
def benchmark_df(df: pd.DataFrame, ks=(1, 5, 10)):
    results = {f"hit@{k}": 0 for k in ks}
    total = len(df)
    exception_len = 0
    for _, row in df.iterrows():
        ground_truth_id = row["_id"]
        # nếu bạn muốn lấy tên từ "title", thì thay "product_name" bằng "title"
        product_name = ' '.join(row["title"].split(' - ')[:-1]) or row.get("product_name")
        if len(product_name) < 4:
            total -= 1
            exception_len += 1
            print(f'error: {product_name}')
            continue
        query = get_random_prompt(str(product_name).lower())
        # print(f"[TEST] id={ground_truth_id} | query='{query}'")
        
        for k in ks:
            retrieved = hybrid_search(query, k)
            id_list = [item["_id"].strip(",") for item in retrieved]
            if ground_truth_id in id_list:
                results[f"hit@{k}"] += 1
                # print(f'{results[f"hit@{k}"]} [TRUE] {ground_truth_id} {k} {query} {retrieved}')
            # else:
            #     print(f'{results[f"hit@{k}"]} [FALSE] {ground_truth_id} {k, query} {retrieved}')
            # if any(r["_id"] == ground_truth_id for r in retrieved):
            #     results[f"hit@{k}"] += 1
        
        # for k in ks:
        #     id_list = [item["_id"].strip(",") for item in retrieved]
        #     if ground_truth_id in id_list:
        #         print(f'[TRUE] {results[f"hit@{k}"]} {ground_truth_id}, {query}')
        #     else:
        #         print('[FALSE]', results[f"hit@{k}"], ground_truth_id, query, retrieved)

    for k in ks:
        results[f"hit@{k}"] /= total # pyright: ignore[reportArgumentType]
    print(f'total: {total}, exception_len: {exception_len}')
    return results

# CSV thực tế
df = pd.read_csv("hoanghamobile.csv")

evals = []

for i in range(20):
    scores = benchmark_df(df, ks=(1, 5, 10))
    evals.append([scores['hit@1'],scores['hit@5'],scores['hit@10']])
    print(f'round {i}: {scores}')

error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
total: 300, exception_len: 20
round 0: {'hit@1': 0.9466666666666667, 'hit@5': 1.0, 'hit@10': 1.0}
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
total: 300, exception_len: 20
round 1: {'hit@1': 0.9533333333333334, 'hit@5': 1.0, 'hit@10': 1.0}
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 
error: 


KeyboardInterrupt: 

In [147]:
def hybrid_search(prompt: str, k: int):
    """
    Trả về list top-k kết quả (dict với _id).
    Bạn sẽ thay bằng search thực tế.
    """
    query_embeddings = model.encode([prompt], prompt_name="query")

    if k > 5 and k % 2 == 0:
        # Compute the (cosine) similarity between the query and document embeddings
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(-similarity.numpy().ravel())
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        tokenized_query = prompt.split(" ")
        result_k = list(bm25.get_top_n(tokenized_query, result_list, n=k))
    
        result_bm25 = [result_k[idx].split()[0] for idx in range(k//2)]
        result_qwen = [np.array(top_similarity_id_products)[_,1] for _ in range(k//2)]
        
        result_combined = [{"_id": np.array(top_similarity_id_products)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        # print(result_combined)
        # result_combined = [{"_id": np.array(result_combined)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        
    else:
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(-similarity.numpy().ravel())
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        result_qwen = [{"_id": np.array(top_similarity_id_products)[_,1], "_score": np.array(top_similarity_id_products)[_,0]} for _ in range(k)]
        result_combined = result_qwen
    
    return result_combined

retrieved = hybrid_search('samsung galaxy a05s có không shop?', 10)
retrieved

[{'_id': np.str_('666baeb69793e149fe739415'), '_score': np.str_('73%')},
 {'_id': np.str_('666baeb69793e149fe739416'), '_score': np.str_('73%')},
 {'_id': np.str_('666baeb49793e149fe7393bc'), '_score': np.str_('69%')},
 {'_id': np.str_('666baeb79793e149fe739471'), '_score': np.str_('64%')},
 {'_id': np.str_('666baeb59793e149fe7393d5'), '_score': np.str_('64%')},
 {'_id': np.str_('666baeb79793e149fe73944d'), '_score': np.str_('62%')},
 {'_id': np.str_('666baeb59793e149fe7393f8'), '_score': np.str_('62%')},
 {'_id': np.str_('666baeb89793e149fe7394b4'), '_score': np.str_('61%')},
 {'_id': np.str_('666baeb79793e149fe73944e'), '_score': np.str_('61%')},
 {'_id': np.str_('666baeb89793e149fe739494'), '_score': np.str_('59%')}]

In [115]:
query_embeddings = model.encode(['có IP khong?'], prompt_name="query")
document_embeddings = model.encode(result_list)
simi = model.similarity(query_embeddings, document_embeddings)

In [120]:
simi

tensor([[0.3426, 0.2391, 0.3169, 0.2672, 0.2045, 0.2916, 0.2926, 0.1928, 0.2588,
         0.3099, 0.2142, 0.2256, 0.2684, 0.2210, 0.2320, 0.2288, 0.2809, 0.2329,
         0.2380, 0.3175, 0.2157, 0.2673, 0.2470, 0.2697, 0.3186, 0.2111, 0.2858,
         0.2706, 0.2524, 0.2873, 0.2200, 0.2922, 0.2394, 0.2243, 0.2313, 0.2781,
         0.3780, 0.3603, 0.2690, 0.2540, 0.3323, 0.3748, 0.2096, 0.1929, 0.3231,
         0.2175, 0.3410, 0.1992, 0.3005, 0.3257, 0.2366, 0.3145, 0.2255, 0.2079,
         0.2194, 0.3367, 0.2211, 0.2134, 0.2975, 0.2177, 0.2541, 0.2613, 0.3260,
         0.3417, 0.2584, 0.2518, 0.2928, 0.3038, 0.2075, 0.2238, 0.3357, 0.3512,
         0.2467, 0.2705, 0.2505, 0.2817, 0.2202, 0.3071, 0.2552, 0.2608, 0.2528,
         0.3232, 0.3225, 0.3201, 0.3209, 0.3259, 0.3206, 0.3219, 0.1824, 0.2774,
         0.2626, 0.3327, 0.3146, 0.3162, 0.3208, 0.3354, 0.3294, 0.2306, 0.2274,
         0.2334, 0.2685, 0.2961, 0.2140, 0.3228, 0.2455, 0.2765, 0.2569, 0.2040,
         0.2690, 0.2801, 0.2

In [116]:
np.argsort(simi)

tensor([[125, 302, 133,  88,   7,  43, 258, 129, 132,  47, 205, 182, 115, 165,
         204, 229, 107,   4, 200, 301,  68,  53,  42, 209, 206,  25,  57, 309,
         102,  10, 164, 141, 212,  20, 231, 150, 202, 218,  45,  59, 110, 254,
         300,  54, 255,  30,  76,  13,  56, 139, 286, 161, 174, 215,  69,  33,
         180, 149, 292,  52,  11, 305,  98, 239, 236, 154,  15, 288, 259, 232,
          97, 167,  34,  14, 224, 131,  17,  99, 112,  50, 166, 293,  18, 186,
         238,   1,  32, 307, 155, 304, 151, 230, 144, 173, 104, 176, 116, 181,
         163,  72, 296,  22, 170, 137, 196, 162, 228, 158,  74, 153,  65, 159,
          28, 237, 156,  80, 187, 179, 222,  39,  60, 299,  78, 252, 106, 127,
         247, 271,  64,   8, 152, 220, 294, 261,  79,  61, 117,  90, 235, 183,
         267, 234, 272, 169, 260, 249,   3,  21, 111, 213, 114, 214,  12, 100,
         197, 184,  38, 108, 219,  23,  73,  27, 188, 251, 194, 201, 145, 185,
         216, 221, 105, 192, 246,  89, 217,  35, 243

In [117]:
simi.shape, np.argmax(simi)

(torch.Size([1, 320]), tensor(313))

In [118]:
len(np.squeeze(np.argsort(simi)))

320

In [125]:

for i in np.squeeze(np.argsort(simi)):
    simi_dict = {'_id': np.squeeze(np.argsort(simi))[i], '_score' : np.squeeze(simi)[i]}
    print(np.squeeze(simi)[i], np.squeeze(np.argsort(simi))[i])

tensor(0.1715) tensor(127)
tensor(0.1749) tensor(276)
tensor(0.1820) tensor(261)
tensor(0.1824) tensor(155)
tensor(0.1928) tensor(129)
tensor(0.1929) tensor(54)
tensor(0.1941) tensor(94)
tensor(0.1982) tensor(8)
tensor(0.1989) tensor(294)
tensor(0.1992) tensor(13)
tensor(0.2000) tensor(314)
tensor(0.2020) tensor(75)
tensor(0.2027) tensor(80)
tensor(0.2030) tensor(201)
tensor(0.2037) tensor(5)
tensor(0.2040) tensor(134)
tensor(0.2040) tensor(158)
tensor(0.2045) tensor(7)
tensor(0.2064) tensor(265)
tensor(0.2066) tensor(273)
tensor(0.2075) tensor(259)
tensor(0.2079) tensor(215)
tensor(0.2096) tensor(300)
tensor(0.2098) tensor(66)
tensor(0.2105) tensor(31)
tensor(0.2111) tensor(25)
tensor(0.2134) tensor(149)
tensor(0.2138) tensor(248)
tensor(0.2140) tensor(170)
tensor(0.2142) tensor(205)
tensor(0.2146) tensor(194)
tensor(0.2148) tensor(234)
tensor(0.2152) tensor(171)
tensor(0.2157) tensor(68)
tensor(0.2160) tensor(317)
tensor(0.2161) tensor(114)
tensor(0.2173) tensor(168)
tensor(0.2175) t

In [126]:
simi_dict

{'_id': tensor(268), '_score': tensor(0.4163)}

In [122]:
simi_dict

{'_id': tensor(268), '_score': tensor(0.4163)}

In [86]:
import numpy as np

arr = np.array([10, 5, 8, 20, 3])

# Sắp xếp giảm dần
sorted_arr = np.sort(arr)[::-1]

sorted_arr

array([20, 10,  8,  5,  3])

In [61]:
import random
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer
import torch

# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-0.6B",
#     device="cuda",
#     model_kwargs={"torch_dtype": "bfloat16"}
# )

# # MODELS EMBEDDING 1
# document_embeddings = model.encode(result_list)

# # MODELS EMBEDDING 2
# tokenized_corpus = [doc.split(" ") for doc in result_list]
# bm25 = BM25Okapi(tokenized_corpus)


# --------------------------
# Prompt templates
# --------------------------
prompts = [
    "{product_name} có không shop?",
    "{product_name} chip gì vậy?",
    "camera con {product_name} thế nào vậy ?",
    "Cho mình hỏi {product_name} được bao nhiêu gb ram vậy?",
    "{product_name} ở HCM còn hàng không?",
    "{product_name} có màu nào và giá bao nhiêu vậy?",
    "Alo tôi muốn đặt mua {product_name}, liên hệ giúp tôi 0985189541.",
    "Top các sản phẩm dở tệ, top1: {product_name} "
]

def get_random_prompt(product_name: str) -> str:
    return random.choice(prompts).format(product_name=product_name)

# --------------------------
# Stub search (replace later)
# --------------------------
def hybrid_search(prompt: str, k: int):
    """
    Trả về list top-k kết quả (dict với _id).
    Bạn sẽ thay bằng search thực tế.
    """
    query_embeddings = model.encode([prompt], prompt_name="query")

    if k > 5 and k % 2 == 0:
        # Compute the (cosine) similarity between the query and document embeddings
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(similarity.numpy().ravel())[:][::-1]
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        tokenized_query = prompt.split(" ")
        result_k = list(bm25.get_top_n(tokenized_query, result_list, n=k))
    
        result_bm25 = [result_k[idx].split()[0] for idx in range(k//2)]
        result_qwen = [np.array(top_similarity_id_products)[_,1] for _ in range(k//2)]
        
        result_combined = result_qwen + result_bm25
        result_combined = [{"_id": result_combined[idx].split()[0]} for idx in range(k)]
        
    else:
        similarity = model.similarity(query_embeddings, document_embeddings)
        top_similarity_idx = np.argsort(similarity.numpy().ravel())[:][::-1]
        top_similarity_id_products = [[f"{int(similarity[0][idx]*100)}%", df.values[idx][0]] for idx in top_similarity_idx]
        
        result_qwen = [{"_id": np.array(top_similarity_id_products)[_,1]} for _ in range(k)]
        result_combined = result_qwen
    
    return result_combined
    
    

# --------------------------
# Benchmark for dataframe
# --------------------------
def benchmark_df(df: pd.DataFrame, ks=(1, 5, 10)):
    results = {f"hit@{k}": 0 for k in ks}
    total = len(df)

    for _, row in df.iterrows():
        ground_truth_id = row["_id"]
        # nếu bạn muốn lấy tên từ "title", thì thay "product_name" bằng "title"
        product_name = row.get("product_name") or row["title"].split("-")[0].strip()

        query = get_random_prompt(product_name)
        query = process_squences(query)
        print(f"[TEST] id={ground_truth_id} | query='{query}'")
        
        for k in ks:
            retrieved = hybrid_search(query, k)
            if any(r["_id"] == ground_truth_id for r in retrieved):
                results[f"hit@{k}"] += 1
        
        print(retrieved)
        
    for k in ks:
        results[f"hit@{k}"] /= total # pyright: ignore[reportArgumentType]

    return results

# CSV thực tế
df = pd.read_csv("hoanghamobile.csv")

scores = benchmark_df(df, ks=(1, 5, 10, 50, 100))

print("\nFinal scores:", scores)

[TEST] id=666baeb49793e149fe7393b4 | query='mình hỏi nokia 3210 4g bao nhiêu gb ram vậy'
[{'_id': '666baeb49793e149fe7393b4'}, {'_id': '666baeb79793e149fe73946b'}, {'_id': '666baeb79793e149fe73946a'}, {'_id': '666baeb79793e149fe739439'}, {'_id': '666baeb99793e149fe7394da'}, {'_id': '666baeb89793e149fe73947c'}, {'_id': '666baeb89793e149fe7394bd'}, {'_id': '666baeb89793e149fe73948d'}, {'_id': '666baeb49793e149fe7393b5'}, {'_id': '666baeb99793e149fe7394db'}, {'_id': '666baeb69793e149fe73942b'}, {'_id': '666baeb89793e149fe7394be'}, {'_id': '666baeb99793e149fe7394c9'}, {'_id': '666baeb89793e149fe73947a'}, {'_id': '666baeb89793e149fe739491'}, {'_id': '666baeb89793e149fe7394bf'}, {'_id': '666baeb89793e149fe73948c'}, {'_id': '666baeb99793e149fe7394d2'}, {'_id': '666baeb99793e149fe7394d9'}, {'_id': '666baeb89793e149fe73948e'}, {'_id': '666baeb99793e149fe7394e7'}, {'_id': '666baeb79793e149fe73943b'}, {'_id': '666baeb89793e149fe7394a4'}, {'_id': '666baeb89793e149fe7394c0'}, {'_id': '666baeb89793e

# model MxbaiRerankV2 (3GB)

https://huggingface.co/mixedbread-ai/mxbai-rerank-large-v2/tree/main


In [73]:
from mxbai_rerank import MxbaiRerankV2

model = MxbaiRerankV2(
    "mixedbread-ai/mxbai-rerank-large-v2"
    )

query = "Có điện thoại iPhone 14 Pro Max không?"
documents = result_list[:]

# Lets get the scores
results = model.rank(query, documents, return_documents=True, top_k=10)

print(results)

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[RankResult(index=195, score=4.8125, document="666baeb89793e149fe739479, https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-14-128gb-chinh-hang-vn-a, điện thoại iphone 14 (128gb) - chính hãng vn/a, - KM 1- Ưu đãi mua combo 3 món củ sạc + dán màn hình + ốp lưng Mophie Zagg chỉ 985.000đ- KM 2- Ưu đãi trả góp 0% qua thẻ tín dụng, Công nghệ màn hình: Super Retina XDR Độ phân giải: 2532 x 1170, 12MP x 12MP, 12MP Kích thước màn hình: 6.1 inch Hệ điều hành: iOS 16 Vi xử lý: A15 Bionic Bộ nhớ trong: 128GB RAM: 6GB Mạng di động: 5G (sub ‑ 6 GHz và mmWave) với 4x4 MIMO8, Gigabit LTE với 4x4 MIMO và LAA8, Chip băng thông siêu rộng cho nhận thức về không gian, NFC với chế độ đọc, Thẻ Express có dự trữ năng lượng Số khe SIM: 1 eSIM, 1 SIM vật lý, 16,290,000 ₫, ['Midnight', 'Blue', 'Purple', 'Red', 'Yellow', 'Starlight']"), RankResult(index=92, score=4.75, document="666baeb69793e149fe739411, https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-15-pro-max-256gb-chinh-hang-vn-a, điện thoại

In [41]:
result_list[0][:512]

"666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng, nan, Công nghệ màn hình: IPS Kích thước màn hình: 2.4 inch Độ phân giải: 2MP Hệ điều hành: S30+ Bộ nhớ trong: 128MB3 RAM: 64MB Mạng di động: 2G, 3G, 4G, Hỗ trợ VoLTE2 Số khe SIM: Hai SIM Nano SIM + Nano SIM Dung lượng pin: 1450mAh, 1,590,000 ₫, ['Màu Vàng', 'Xanh', 'Màu Đen']"

In [44]:
question = result_list[0][:512]
text = result_list[0][:512]

text, question

("666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng, nan, Công nghệ màn hình: IPS Kích thước màn hình: 2.4 inch Độ phân giải: 2MP Hệ điều hành: S30+ Bộ nhớ trong: 128MB3 RAM: 64MB Mạng di động: 2G, 3G, 4G, Hỗ trợ VoLTE2 Số khe SIM: Hai SIM Nano SIM + Nano SIM Dung lượng pin: 1450mAh, 1,590,000 ₫, ['Màu Vàng', 'Xanh', 'Màu Đen']",
 "666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng, nan, Công nghệ màn hình: IPS Kích thước màn hình: 2.4 inch Độ phân giải: 2MP Hệ điều hành: S30+ Bộ nhớ trong: 128MB3 RAM: 64MB Mạng di động: 2G, 3G, 4G, Hỗ trợ VoLTE2 Số khe SIM: Hai SIM Nano SIM + Nano SIM Dung lượng pin: 1450mAh, 1,590,000 ₫, ['Màu Vàng', 'Xanh', 'Màu Đen']")

# model (440MB)


# Model tàu lau (google-bert/bert-base-uncased)


In [72]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("google-bert/bert-base-uncased")

question = "Thằng Vinh là ai?"
text = "Vinh là một thằng rất chịu cực, chịu khó, chịu chơi"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# target is "nice puppet"
target_start_index = torch.tensor([answer_start_index])
target_end_index = torch.tensor([answer_end_index-1])

outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
print(round(loss.item(), 2))

import numpy as np

start_idx = np.argmax(outputs.start_logits.detach().numpy()).item()  # 14
end_idx = np.argmax(outputs.end_logits.detach().numpy()).item()    # 15

text[start_idx:end_idx]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3.25


'h là một thằng '

In [74]:
result_list

["666baeb49793e149fe7393b4, https://hoanghamobile.com/dien-thoai/nokia-3210-4g-chinh-hang, nokia 3210 4g - chính hãng, nan, Công nghệ màn hình: IPS Kích thước màn hình: 2.4 inch Độ phân giải: 2MP Hệ điều hành: S30+ Bộ nhớ trong: 128MB3 RAM: 64MB Mạng di động: 2G, 3G, 4G, Hỗ trợ VoLTE2 Số khe SIM: Hai SIM Nano SIM + Nano SIM Dung lượng pin: 1450mAh, 1,590,000 ₫, ['Màu Vàng', 'Xanh', 'Màu Đen']",
 "666baeb49793e149fe7393bc, https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-a05s-6gb-128gb-bh%C4%91t, samsung galaxy a05s - 6gb/128gb (bhđt), - Ưu đãi trả góp 0% qua Shinhan Finance hoặc Mirae Asset Finance- Giảm 5% không giới hạn khuyến mãi qua Homepaylater- Giảm thêm tới 700.000đ khi thanh toán qua Kredivo.- Giảm 50% tối đa 700k khi mở thẻ tín dụng Vpbank trên SenID- Giảm 20% tối đa 500k khi mở thẻ tín dụng TPBank EVO- Mở thẻ tín dụng VIB - Nhận Voucher 600.000đ- Giảm 1% tối đa 100.000đ khi thanh toán qua Zalopay, Công nghệ màn hình: PLS LCD, 90Hz Độ phân giải: FHD+ (2400 x 1080),

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(result_list)
print(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.03773434  0.06983994 -0.00354255 ... -0.00576472 -0.04541253
  -0.00612617]
 [-0.07201083  0.08397817 -0.06605534 ... -0.00744218 -0.06556925
   0.00879733]
 [-0.03688413  0.03849854 -0.03727051 ... -0.00699501 -0.04964436
   0.03263906]
 ...
 [-0.04679155 -0.01236879 -0.00297951 ...  0.0126444  -0.04404563
   0.04448592]
 [-0.04058863 -0.00324071 -0.0157772  ...  0.00984802 -0.04960276
   0.0418218 ]
 [-0.03446696  0.00395076 -0.01227578 ... -0.0105386  -0.05219663
   0.03908668]]


In [ ]:


from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 0.02250261 -0.07829168 -0.02303072 ... -0.00827929  0.02652685
  -0.00201898]
 [ 0.04170232  0.00109744 -0.0155342  ... -0.0218163  -0.06359362
  -0.00875289]]
